In [1]:
import torch
import os
import torch.nn as nn
import random
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from torch.utils.tensorboard import SummaryWriter

In [2]:
SEED = 100

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
writer = SummaryWriter()

In [5]:
def getNameList(path):
    return os.listdir(path)

def resizeImage(input_path, output_path):
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    nameList = getNameList(input_path)
        
    for dirIdx in range(len(nameList)):
        dir_name = nameList[dirIdx]
        out_dir = os.path.join(output_path, dir_name)
        
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        
        dir_path = os.path.join(input_path, dir_name)
            
        dir_images = os.listdir(dir_path)
        num_images = len(dir_images)
        print("idx : " + str(dirIdx) + ', len : ' + str(num_images))
            
        for i,image in enumerate(dir_images):
            with open(os.path.join(dir_path, image), 'r+b') as f:
                with Image.open(f) as img:
                    img = img.resize([256, 256], Image.ANTIALIAS)
                    img.save(os.path.join(out_dir, image), img.format)
    
    return nameList

def createMap(ylist):
    index2name = {}
    name2index = {}
    for i in range(len(ylist)):
        name = ylist[i]
        index2name[i] = name
        name2index[name] = i
    
    return index2name, name2index
        

In [6]:
y_dataList = resizeImage("./Dog_breeds_Dataset", "./Dog_breeds_Dataset_resized")

idx : 0, len : 130
idx : 1, len : 130
idx : 2, len : 130
idx : 3, len : 129
idx : 4, len : 131
idx : 5, len : 131
idx : 6, len : 129
idx : 7, len : 125
idx : 8, len : 129
idx : 9, len : 130
idx : 10, len : 130
idx : 11, len : 129
idx : 12, len : 127
idx : 13, len : 133
idx : 14, len : 130
idx : 15, len : 131
idx : 16, len : 129
idx : 17, len : 129
idx : 18, len : 124
idx : 19, len : 129
idx : 20, len : 130
idx : 21, len : 127
idx : 22, len : 128
idx : 23, len : 130
idx : 24, len : 131


In [7]:
index2name, name2index = createMap(y_dataList)
print(y_dataList)
print(len(y_dataList))
print()
print()
print(index2name[3], name2index["standard_schnauzer"])

['African_hunting_dog', 'basenji', 'beagle', 'black-and-tan_coonhound', 'Blenheim_spaniel', 'Boston_bull', 'Chihuahua', 'clumber', 'dingo', 'EntleBucher', 'Japanese_spaniel', 'keeshond', 'komondor', 'Leonberg', 'Maltese_dog', 'Norwegian_elkhound', 'otterhound', 'Pomeranian', 'pug', 'Shih-Tzu', 'Siberian_husky', 'silky_terrier', 'standard_schnauzer', 'Welsh_springer_spaniel', 'Yorkshire_terrier']
25


black-and-tan_coonhound 22


In [8]:
class customDS(data.Dataset):
    def __init__(self, yList, root, name2idx, transform=None):
        self.yList = yList
        self.root = root
        self.x_data = []
        self.y_data = []
        self.transform = transform
        self.name2idx = name2idx

        for i in range(len(self.yList)):
            path = os.path.join(root, self.yList[i])
            
            images = os.listdir(path)
            for j in range(len(images)):
                self.x_data.append(images[j])
                self.y_data.append(self.yList[i])    
            
    def __getitem__(self, index):
        target = self.y_data[index]
        
        image = Image.open(os.path.join(self.root, target, self.x_data[index])).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        
        target = self.name2idx[target]
        
        return image, target
    
    def __len__(self):
        return len(self.y_data)

In [9]:
transform = transforms.Compose([
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

totalDS = customDS(y_dataList, "./Dog_breeds_Dataset_resized", name2index, transform=transform)

In [10]:
print(len(totalDS))

3231


In [11]:
def makeValRandom():
    
    idxList = list(range(len(totalDS)))
    np.random.shuffle(idxList)

    split = int(np.floor(0.7 * len(totalDS)))
    trainIdx, validIdx = idxList[:split], idxList[split:]

    train_sampler = SubsetRandomSampler(trainIdx)
    valid_sampler = SubsetRandomSampler(validIdx)
    
    train_loader = DataLoader(
        dataset=totalDS,
        batch_size=50,
        num_workers=0,
        sampler=train_sampler
    )
    
    val_loader = DataLoader(
        dataset=totalDS,
        batch_size=50,
        num_workers=0,
        sampler=valid_sampler
    )
    
    return train_loader, val_loader

In [12]:
train_loader, val_loader = makeValRandom()

In [13]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            #224x224 -> 
            nn.Conv2d(3, 64, 4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1, bias=False),
            nn.LeakyReLU(),
            nn.Conv2d(128, 256, 3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(256, 512, 3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(),
            nn.MaxPool2d(2,2),
        )
        
        self.fc = nn.Linear(512 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 25)
        self.dropout = nn.Dropout(p=0.4)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = x.view(-1, 3, 224, 224)
        out = self.conv(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = self.dropout(F.relu(out))
        out = self.fc2(out)
        return out

In [14]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
epochs = 50
#train_loader, val_loader = makeValRandom()

for epoch in range(1, epochs + 1):
    
    trn_loss = 0
    trn_total = 0
    trn_correct = 0
    
    model.train()
    for img, label in train_loader:
        
        img, label = img.to(device), label.to(device)
        
        hypothesis = model(img)
        loss = criterion(hypothesis, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        trn_loss += loss.item()
        
        _, predicted = hypothesis.max(1)
        trn_total += label.size(0)
        trn_correct += predicted.eq(label).sum().item()
    trn_acc = trn_correct / trn_total
    
    val_loss = 0
    val_total = 0
    val_correct = 0
    
    model.eval()
    with torch.no_grad():
        for img, label in val_loader:
            img, label = img.to(device), label.to(device)
            
            results = model(img)
            loss = criterion(results, label)
            
            val_loss += loss.item()
            
            _, predicted = results.max(1)
            val_total += label.size(0)
            val_correct += predicted.eq(label).sum().item()
        val_acc = val_correct / val_total
        
        writer.add_scalar('Loss/train_loss', trn_loss, epoch)
        writer.add_scalar('Loss/valid_loss', val_loss, epoch)
        writer.add_scalar('Accuracy/train_accuracy', trn_acc * 100, epoch)
        writer.add_scalar('Accuracy/valid_accuracy', val_acc * 100, epoch)
        
        print('epoch : (%d/%d) tr_Loss: %.2f, val_Loss: %.2f / tr_Acc: %.1f, Val_Acc: %.1f'
                 %(epoch, epochs, trn_loss, val_loss, trn_acc*100, val_acc*100))

epoch : (1/50) tr_Loss: 160.59, val_Loss: 64.15 / tr_Acc: 4.5, Val_Acc: 6.7
epoch : (2/50) tr_Loss: 146.67, val_Loss: 62.18 / tr_Acc: 7.2, Val_Acc: 9.1
epoch : (3/50) tr_Loss: 141.52, val_Loss: 60.51 / tr_Acc: 9.8, Val_Acc: 11.0
epoch : (4/50) tr_Loss: 135.68, val_Loss: 58.61 / tr_Acc: 13.4, Val_Acc: 14.8
epoch : (5/50) tr_Loss: 131.10, val_Loss: 56.23 / tr_Acc: 16.5, Val_Acc: 17.6
epoch : (6/50) tr_Loss: 125.30, val_Loss: 54.55 / tr_Acc: 19.0, Val_Acc: 21.3
epoch : (7/50) tr_Loss: 120.36, val_Loss: 53.16 / tr_Acc: 22.0, Val_Acc: 20.5
epoch : (8/50) tr_Loss: 118.11, val_Loss: 51.52 / tr_Acc: 23.6, Val_Acc: 22.9
epoch : (9/50) tr_Loss: 112.78, val_Loss: 52.36 / tr_Acc: 27.2, Val_Acc: 23.6
epoch : (10/50) tr_Loss: 110.29, val_Loss: 48.88 / tr_Acc: 27.9, Val_Acc: 29.8
epoch : (11/50) tr_Loss: 103.42, val_Loss: 48.76 / tr_Acc: 32.7, Val_Acc: 27.0
epoch : (12/50) tr_Loss: 101.22, val_Loss: 47.02 / tr_Acc: 34.1, Val_Acc: 29.3
epoch : (13/50) tr_Loss: 97.65, val_Loss: 46.96 / tr_Acc: 35.8, Va

In [16]:
writer.close()